In [1]:
import numpy as np
from random import uniform
from timeit import timeit
import numexpr as ne

class Particle:
    def __init__(self, x, y, velocidad_angular):
        self.x = x
        self.y = y
        self.velocidad_angular = velocidad_angular

class ParticleSimulator:

    def __init__(self, particles):
        self.particles = particles

    def evolve_python(self, dt):
        timestep = 0.00001
        nsteps = int(dt / timestep)
        for i in range(nsteps):
            for p in self.particles:
                norm = (p.x**2 + p.y**2)**0.5
                v_x = (-p.y) / norm
                v_y = p.x / norm
                d_x = timestep * p.velocidad_angular * v_x
                d_y = timestep * p.velocidad_angular * v_y
                p.x += d_x
                p.y += d_y

    def evolve_numpy(self, dt):
        timestep = 0.00001
        nsteps = int(dt / timestep)
        r_i = np.array([[p.x, p.y] for p in self.particles])
        velocidad_angular_i = np.array([p.velocidad_angular for p in self.particles])
        for i in range(nsteps):
            norm_i = np.sqrt((r_i ** 2).sum(axis=1))
            v_i = r_i[:, [1, 0]] * np.array([-1, 1]) / norm_i[:, np.newaxis]
            d_i = timestep * velocidad_angular_i[:, np.newaxis] * v_i
            r_i += d_i
        for i, p in enumerate(self.particles):
            p.x, p.y = r_i[i]


def benchmark(npart=1000, method='python'):
    particles = [Particle(uniform(-1.0, 1.0), uniform(-1.0, 1.0), uniform(-1.0, 1.0)) for _ in range(npart)]
    simulator = ParticleSimulator(particles)
    if method == 'python':
        simulator.evolve_python(0.1)
    elif method == 'numpy':
        simulator.evolve_numpy(0.1)

In [2]:
if __name__ == "__main__":
    # Example with 1000 particles
    %timeit benchmark(npart=1_000, method='python')
    %timeit benchmark(npart=1_000, method='numpy')


    # Example with 10000 particles
    %timeit benchmark(npart=10_000, method='python')
    %timeit benchmark(npart=10_000, method='numpy')

9.09 s ± 1.03 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
575 ms ± 51.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1min 30s ± 1.93 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
3.84 s ± 310 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
dt1 = np.arange(1e6)
dt2 = np.arange(1e6)

%timeit 2*dt1 + 3*dt2
%timeit ne.evaluate("2*dt1 + 3*dt2")

6.38 ms ± 115 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
3.4 ms ± 83.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [5]:
import numpy as np
import numexpr as ne
from timeit import timeit

# Define the arrays with a million elements each
dt1 = np.arange(1e6)
dt2 = np.arange(1e6)

# Define the operations to be performed
def pure_python():
    return [2*x + 3*y for x, y in zip(dt1, dt2)]

def numpy_array():
    return 2*dt1 + 3*dt2

def numexpr_expr():
    return ne.evaluate("2*dt1 + 3*dt2")

# Benchmark the execution time for each approach
n_trials = 1000  # Number of trials for timeit
time_python = timeit(pure_python, number=n_trials)
time_numpy = timeit(numpy_array, number=n_trials)
time_numexpr = timeit(numexpr_expr, number=n_trials)

# Display the results
print(f"Pure Python execution time: {time_python} seconds")
print(f"NumPy execution time: {time_numpy} seconds")
print(f"NumExpr execution time: {time_numexpr} seconds")


Pure Python execution time: 432.1467014309999 seconds
NumPy execution time: 2.765671139999995 seconds
NumExpr execution time: 3.3016093899998396 seconds
